In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
originImage = cv2.imread("lena.jpg")
grayImage = cv2.cvtColor(originImage,cv2.COLOR_BGR2GRAY)
originImage32x32 = cv2.imread("lena_32x32.jpg")
grayImage32x32 = cv2.cvtColor(originImage32x32,cv2.COLOR_BGR2GRAY)
plt.subplot(1, 3, 1)
plt.title("full")
plt.imshow(originImage)
plt.subplot(1, 3, 2)
plt.title("gray")
plt.gray()
plt.imshow(grayImage)

In [ ]:
# 按照src图逐个像素计算个数.
# 灰度像素范围0~255.
# 返回灰度直方图统计
def calcHistgram(src):
    w,h = src.shape
    grayHist=np.zeros([256],np.uint64)
    for r in range(h):
        for c in range(w):
            grayHist[src[r,c]]+=1
    
    return grayHist


#### two peaks' segmentation

> the 1st peak
find the max peak

> the 2ed peak

$$peak_{second}=argmax_{k=0}^{K=255}(k-peak_{frist})^{2}*histgram[k]$$

In [2]:
def calcTwoPeaksAndThreash(src):
    # 得到hist统计
    histgram=calcHistgram(src)
    # 找到hist直方图中第一个最大峰值的像素值.
    maxLoc=np.where(histgram==np.max(histgram))
    # np.where返回相等的index.这里index也就是像素值.范围0~255
    firstPeak=maxLoc[0][0]
    # 计算第二个峰值
    measureDists=np.zeros([256],np.float32)
    for k in range(256):
        measureDists[k]=(k-firstPeak)**2*histgram[k]
    maxLoc2=np.where(measureDists==np.max(measureDists))
    secondPeak=maxLoc2[0][0]
    # 找到两个波峰间的最小值作为threashold.
    threash=0
    if firstPeak > secondPeak: # 第一个峰值在第二个峰值的右边.
        partHist=histgram[secondPeak:firstPeak]
        minLoc=np.where(partHist==np.min(partHist))
        threash=secondPeak+minLoc[0][0]+1
    else:
        partHist=histgram[firstPeak:secondPeak]
        minLoc=np.where(partHist==np.min(partHist))
        threash=firstPeak+minLoc[0][0]+1
    # 找到阈值后进行二分处理
    threahImage_out = src.copy()
    threahImage_out[threahImage_out>threash]=255
    threahImage_out[threahImage_out<=threash]=0
    return (threash,threahImage_out)
        
    

In [3]:
threash,threah_out=calcTwoPeaksAndThreash(grayImage)
print("threah:",threash)
plt.subplot(1, 1, 1)
plt.title("threash image")
plt.imshow(threah_out)

NameError: name 'grayImage' is not defined

### entropy segmentation

1.信息熵定义

$$entropy(u)=-\sum _{i=1}^{N}p_ilogp_i$$

2.概率直方图

$$normHist=\frac{grayHist}{w*h}$$

3.累加概率直方图

$$cumeHist(k)= \sum_{i=0}^{k}normHist_I(i),k \in [0,255]$$

4.各个灰度级熵

$$entropy(t)=-\sum_{k=0}^{t}normHist_I(t)*log_{10}(normHist_I(t)), 0\leq t\leq 255$$

5.f1(x)和f2(x)

$$f_1(t)=\frac{entropy(t)}{entropy(255)}*\frac{log_{10}(cumuHist(t))}{log_{10}(max(cumuHist(0),cumuHist(1),...,cumuHist(t)))}$$

$$f_2(t)=(1-\frac{entropy(t)}{entropy(255)})*\frac{log_{10}(1-cumuHist(t))}{log_{10}(max(cumuHist(t+1),cumuHist(t+2),...,cumuHist(255)))}$$

In [5]:
def calcNormHist(histgram,w,h):
    return histgram/(float(w*h))

def calcCumuNormHist(normhist):
    cumuHist=np.zeros([256],np.float32)
    for k in range(256):
        if k ==0:
            cumuHist[k]=normhist(k)
        else:
            cumuHist[k]=cumuHist[k-1]+normhist[k]
    
    return cumuHist

def calcGrayscaleEntropy(normhist):
    grayscaleEnt=np.zeros([256],np.float32)
    for k in range(256):
        if k==0:
            if normhist[k]==0:
                grayscaleEnt[k]=0
            else:
                grayscaleEnt[k]=-normhist[k]*math.log10(normhist[k])
        else:
            if normhist[k]==0:
                grayscaleEnt[k]=grayscaleEnt[k-1]
            else:
                grayscaleEnt[k]=grayscaleEnt[k-1]-normhist[k]*math.log10(normhist[k])
    return grayscaleEnt

In [6]:
def threashEntropy(src):
    histgram=calcHistgram(src)
    normHist=calcNormHist(histgram)
    cumuHist=calcCumuNormHist(normHist)
    grayscaleEnt = calcGrayscaleEntropy(normHist)
    # 计算ft1和ft2
    fT=np.zeros([256],np.float32)
    ft1,ft2=0.0,0.0
    for k in range(255): # 遍历k,每次k作为ft1和ft2分割点.
        maxFront=np.max(cumuHist[0:k+1])
        maxBack=np.max(cumuHist[k+1:256])
        if(maxFront==0 or maxFront==1 or cumuHist[k]==0 or cumuHist[k]==1 or entropy[255]==0):
            ft1=0.0
        else:
            ft1 = entropy[k]/entropy[255]*(math.log10(cumuHist[k])/math.log10(maxFront))
        
        if(maxBack==0 or maxBack==1 or cumuHist[k]==0 or cumuHist[k]==1 or entropy[255]==0):
            ft2 = 0.0
        else:
            ft2 = (1-entropy[k])/entropy[255]*(math.log10(1-cumuHist[k])/math.log10(maxBack))
        
        fT[k]=ft1+ft2
    
    # 找到最大值索引
    
    threashLoc=np.where(fT==np.max(fT))
    thresh=threashLoc[0][0]
    
    # 根据阈值做图像分割
    
    threshhold_image = src.copy()
    threshhold_image[threshhold_image>thresh]=255
    threshhold_image[threshhold_image<=thresh]=0
    
    return threshhold_image



### ostu 

1.误差定义
$$mean=oneCumuHist[255]$$
$$\sigma ^2(k)=\frac{(mean*zeroCumuHist(k)-oneCumuHist(k))^2}{zeroCumuHist(k)*(1-zeroCumuHist(k))}$$

2.找到最大误差的那个灰度级,依次作为阈值,对图像分割.
$$threash=arg_{k\in [0,255)}max(\sigma ^2(k))$$

In [ ]:

def calcZeroAndOneCumuNormHist(normhist):
    zeroCumuHist=np.zeros([256],np.float32)
    oneCumuHist=np.zeros([256],np.float32)
    for k in range(256):
        if k ==0:
            zeroCumuHist[k]=normhist(k)
            oneCumuHist[k]=(k)*normhist(k)
        else:
            zeroCumuHist[k]=zeroCumuHist[k-1]+normhist[k]
            oneCumuHist[k]=oneCumuHist[k-1]+(k)*normhist[k]
    
    return zeroCumuHist,oneCumuHist

def ostu(src):
    histgram=calcHistgram(src)
    normHist=calcNormHist(histgram)
    zeroCumuHist,oneCumuHist=calcZeroAndOneCumuNormHist(normHist) # 计算得到zero和one的cumulative的概率分布
    # 计算类间方差
    variance=np.zeros([256],np.float32)
    for k in range(255):
        if zeroCumuHist[k]==0 or zeroCumuHist[k]==1:
            variance[k]=0
        else:
            variance[k]=math.pow(oneCumuHist[255]*zeroCumuHist[k]-oneCumuHist[k],2)/(zeroCumuHist[k]*(1-zeroCumuHist[k]))

    # 找到阈值
    varLoc=np.where(variance=np.max(variance))
    thresh=varLoc[0][0]
    # 切分
    
    thresh_img=src.copy()
    thresh_img[thresh_img>thresh]=255
    thresh_img[thresh_img<=thresh]=0
    
    return (threash,thresh_img)


### 自适应阈值分割

凭经验,平滑窗口的宽度应该大于待识别物体的宽度.平滑尺寸越大,越能更好的作为每个像素的参考.

In [ ]:
def adaptiveThreash(src,winsize,ratio=0.15):
    I_mean=cv2.boxFilter(src,cv2.CV_32FC1,winsize)
    out=src-(1-ratio)*I_mean
    out[out>=0]=255
    out[out<0]=0
    out=out.astype(np.uint8)
    return out